# ¡Hola, Diana!  

Mi nombre es Carlos Ortiz, soy code reviewer de TripleTen y voy a revisar el proyecto que acabas de desarrollar.

Cuando vea un error la primera vez, lo señalaré. Deberás encontrarlo y arreglarlo. La intención es que te prepares para un espacio real de trabajo. En un trabajo, el líder de tu equipo hará lo mismo. Si no puedes solucionar el error, te daré más información en la próxima ocasión. 

Encontrarás mis comentarios más abajo - **por favor, no los muevas, no los modifiques ni los borres**.

¿Cómo lo voy a hacer? Voy a leer detenidamente cada una de las implementaciones que has llevado a cabo para cumplir con lo solicitado. Verás los comentarios de esta forma:

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Si todo está perfecto.
</div>


<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>


<div class="alert alert-block alert-danger">
    
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
    
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>


Puedes responderme de esta forma: 


<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class="tocSkip"></a>
</div>
¡Empecemos!

# Entrenamiento de un modelo para analizar las terminación de contratos de los clientes de Beta Bank

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Carga de liberías y datos](#data_review)
* [Preparación de datos](#seg)
* [Análisis de equilibrio de clases](class)
    * [Parámetro class_weight](#we)
    * [Sobremuestreo](#up)
    * [Submuestreo](#down)
* [¿Qué modelo es mejor?](#train)
    * [Árbol de decisión](#tree)
    * [Bosque aleatorio](#forest)
    * [Regresión logística](#reg)
* [Calidad del modelo](#cal)
* [Conclusión general](#conclusion)

## Introducción <a id='intro'></a>

El banco Beta Bank necesita predecir si un cliente dejará el banco pronto y se tiene los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.
Se creará un modelo con el máximo valor F1 posible, de al menos 0.59 y se verificará el F1 para el conjunto de prueba. 
Además, se medirá la métrica AUC-ROC para compararla con el valor F1.

## Carga de librerias y datos <a id='data_review'></a>

In [1]:
# Cargar todas las librerías
import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Buen trabajo con la importación de datos y de librerías.
</div>

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


- La característica Tenure tiene datos ausentes
- Los nombres de las columnas no están en formato snake_case.
- La característica que necesitamos predecir es Exited, lo cual será el objetivo. 
- Las demás columnas serán las observaciones para predecir el objetivo.
- El objetivo es categórico ya que se predecirá si el cliente dejará o no el banco.

## Preparación de datos <a id='seg'></a>

- Se cambiará el nombre de las columnas al formato snake_case.

In [5]:
df.columns = df.columns.str.lower()

In [6]:
df.columns 

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [7]:
df = df.rename(columns={'rownumber': 'row_number', 'customerid': 'customer_id', 'creditscore':'credit_score',
                       'numofproducts': 'num_of_products', 'hascrcard': 'credit_card', 'isactivemember':'active_member',
                       'estimatedsalary':'estimated_salary'})

In [8]:
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'credit_card',
       'active_member', 'estimated_salary', 'exited'],
      dtype='object')

- Se imputarán los datos ausentes en Tenure con la mediana de la edad para no sesgar los datos.
- Se transformará características categóricas en características numéricas mediante one-hot (OHE) usando la función get_dummies().

In [9]:
filter = df[df['tenure'].isnull()]
filter

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,credit_card,active_member,estimated_salary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [10]:
filter['exited'].value_counts()

0    726
1    183
Name: exited, dtype: int64

In [11]:
age_median = df.groupby('age')['tenure'].median() 

In [12]:
 def tenure_fixed(row):
    age = row['age']
    if pd.isna(row['tenure']):
        return age_median[age]
    else:
        return row['tenure']

In [13]:
df['tenure'] = df.apply(tenure_fixed, axis=1)

In [14]:
#comprobacion
df['tenure'].isna().sum()

0

In [15]:
data_ohe = pd.get_dummies(df, drop_first=True)

- Determinamos las varibles de observacion y objetivo para cada conjunto de datos.
- Como no contamos con un conjunto de prueba, se dividirán los datos fuente en proporción 3:1:1, un 60% para el conjunto de entrenamiento, 20% para el conjunto de validación y 20% para el conjunto de prueba.

In [16]:
df_train, df_valid_test = train_test_split(data_ohe, test_size = 0.4, random_state = 12233)

In [17]:
df_valid, df_test = train_test_split(df_valid_test, test_size = 0.5, random_state = 12233)

In [18]:
#variables para las características y para la característica objetivo 

#entrenamiento
features_train = df_train.drop(['exited'], axis=1)
target_train = df_train['exited']
#validacion
features_valid = df_valid.drop(['exited'], axis=1)
target_valid = df_valid['exited']
#prueba
features_test = df_test.drop(['exited'], axis=1)
target_test = df_test['exited']

Para que el algoritmo no crea que las características con mayores magnitudes y dispersión son más importantes, se estandarizará los datos con StandardScaler().

In [19]:
numeric = ['row_number', 'customer_id', 'credit_score',
       'age', 'tenure', 'balance',
       'num_of_products', 'credit_card','active_member', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Análisis de equilibrio de clases <a id='class'></a>

### Prueba sin técnicas de equilibrio de clases <a id='tree'></a>

Modelo sin equilibrio de clases, el valor F1 es muy bajo.

#### Modelo regresión logística <a id='reg'></a>

In [20]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.3256637168141593


#### Modelo árbol de decisión <a id='tree'></a>

In [21]:
best_score = 0
best_depth = 0
for depth in range(1, 5):
        model = DecisionTreeClassifier(random_state = 12233, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        score = f1_score(target_valid, predictions_valid)
        
        if score > best_score:
            best_score = score
            best_depth = depth

print("F1 del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_score))

F1 del mejor modelo en el conjunto de validación (depth = 2): 0.5027472527472528


#### Modelo bosque aleatorio <a id='forest'></a>

In [22]:
best_score = 0
best_est = 0
for est in range(1, 100):
    model = RandomForestClassifier(random_state=12233, n_estimators=est)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_est = est

print("F1 del mejor modelo en el conjunto de validación (n_estimators ={}): {}".format(best_est, best_score))

F1 del mejor modelo en el conjunto de validación (n_estimators =93): 0.5144927536231885


In [52]:
best_score = 0
best_split = 0
for split in range(2, 20):
    model = RandomForestClassifier(random_state=12233, min_samples_split=split, n_estimators=93)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_split = split

print("F1 del mejor modelo en el conjunto de validación (min_samples_split ={}): {}".format(best_split, best_score))

F1 del mejor modelo en el conjunto de validación (min_samples_split =2): 0.5144927536231885


El modelo que mejor resultado dio sin el equilibrio de clases fue el bosque aleatorio y arból de decisión.

### Parámetro class_weight <a id='we'></a>

#### Modelo regresión logística <a id='reg'></a>

In [24]:
model = LogisticRegression(random_state=12345, class_weight = 'balanced', solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.46673095467695275


#### Modelo árbol de decisión <a id='tree'></a>

In [25]:
best_score = 0
best_depth = 0
for depth in range(1, 5):
        model = DecisionTreeClassifier(random_state = 12233, class_weight = 'balanced', max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        score = f1_score(target_valid, predicted_valid)
        
        if score > best_score:
            best_score = score
            best_depth = depth

print("F1 del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_score))

F1 del mejor modelo en el conjunto de validación (depth = 4): 0.5441176470588236


#### Modelo bosque aleatorio <a id='forest'></a>

In [26]:
best_score = 0
best_est = 0
for est in range(1, 100):
    model = RandomForestClassifier(random_state=12233, class_weight = 'balanced', n_estimators=est)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_est = est

print("F1 del mejor modelo en el conjunto de validación (n_estimators ={}): {}".format(best_est, best_score))

F1 del mejor modelo en el conjunto de validación (n_estimators =65): 0.47602131438721135


In [53]:
best_score = 0
best_split = 0
for split in range(2, 20):
    model = RandomForestClassifier(random_state=12233, class_weight = 'balanced', 
                                   min_samples_split=split, n_estimators=65)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_split = split

print("F1 del mejor modelo en el conjunto de validación (min_samples_split ={}): {}".format(best_split, best_score))

F1 del mejor modelo en el conjunto de validación (min_samples_split =17): 0.6207827260458839


El hiperparámetro min_samples_split auemntó el valor F1.

### Sobremuestreo <a id='up'></a>

- Con una función se probara la técnica de sobremuestreo para el equilibrio de clases. 

In [28]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled

In [29]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

#### Modelo regresión logística <a id='reg'></a>

In [30]:
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.4217298070050036


#### Modelo árbol de decisión <a id='tree'></a>

In [31]:
best_score = 0
best_depth = 0
for depth in range(1, 5):
        model = DecisionTreeClassifier(random_state = 12233, max_depth=depth)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        score = f1_score(target_valid, predicted_valid)
        
        if score > best_score:
            best_score = score
            best_depth = depth

print("F1 del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_score))

F1 del mejor modelo en el conjunto de validación (depth = 3): 0.422184493491794


#### Modelo bosque aleatorio <a id='forest'></a>

In [32]:
best_score = 0
best_est = 0
for est in range(1, 65):
    model = RandomForestClassifier(random_state=12233, n_estimators=est)
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_est = est

print("F1 del mejor modelo en el conjunto de validación (n_estimators ={}): {}".format(best_est, best_score))

F1 del mejor modelo en el conjunto de validación (n_estimators =37): 0.5895316804407713


In [54]:
best_score = 0
best_split = 0
for split in range(2, 20):
    model = RandomForestClassifier(random_state=12233, min_samples_split=split, n_estimators=37)
    model.fit(features_upsampled, target_upsampled) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_split = split

print("F1 del mejor modelo en el conjunto de validación (min_samples_split ={}): {}".format(best_split, best_score))

F1 del mejor modelo en el conjunto de validación (min_samples_split =11): 0.624390243902439


El hiperparámetro min_samples_split auementó el valor F1.

### Submuestreo <a id='down'></a>

- Con una función se probara la técnica de submuestreo para el equilibrio de clases. 

In [34]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

In [35]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

#### Modelo regresión logística <a id='reg'></a>

In [36]:
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.39977851605758585


#### Modelo árbol de decisión <a id='tree'></a>

In [37]:
best_score = 0
best_depth = 0
for depth in range(1, 5):
        model = DecisionTreeClassifier(random_state = 12233, max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        score = f1_score(target_valid, predicted_valid)
        
        if score > best_score:
            best_score = score
            best_depth = depth

print("F1 del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_score))

F1 del mejor modelo en el conjunto de validación (depth = 3): 0.4378346222486616


#### Modelo bosque aleatorio <a id='forest'></a>

In [38]:
best_score = 0
best_est = 0
for est in range(1, 100):
    model = RandomForestClassifier(random_state=12233, class_weight = 'balanced', n_estimators=est)
    model.fit(features_downsampled, target_downsampled)
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_est = est

print("F1 del mejor modelo en el conjunto de validación (n_estimators ={}): {}".format(best_est, best_score))

F1 del mejor modelo en el conjunto de validación (n_estimators =4): 0.4279007377598927


In [55]:
best_score = 0
best_split = 0
for split in range(2, 20):
    model = RandomForestClassifier(random_state=12233, min_samples_split=split, n_estimators=4)
    model.fit(features_downsampled, target_downsampled) 
    predictions_valid = model.predict(features_valid)  
    score = f1_score(target_valid, predictions_valid)
    
    if score > best_score:
        best_score = score
        best_split = split

print("F1 del mejor modelo en el conjunto de validación (min_samples_split ={}): {}".format(best_split, best_score))

F1 del mejor modelo en el conjunto de validación (min_samples_split =2): 0.4147764095917045


- La que mejor resultado dio fue incluir el parametro class_weight en balanced y el sobremuestreo.
- El hiperparámetro min_samples_split y n_estimators mejoraron la metrica del modelo.

## Calidad del modelo <a id='cal'></a>

### Sin equilibrio de clases

#### Modelo regresión logística 

In [40]:
model = LogisticRegression(random_state=12345,solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.48013245033112584
Test set: 0.3182640144665461
AUC_ROC
Test set: 0.5015487606753861


#### Modelo árbol de decisión 

In [41]:
model = DecisionTreeClassifier(random_state=12233, max_depth = 2)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.5128205128205128
Test set: 0.5149359886201992
AUC_ROC
Test set: 0.5233491475494814


#### Modelo bosque aleatorio

In [56]:
model = RandomForestClassifier(random_state=12233, min_samples_split = 2, n_estimators=93)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 1.0
Test set: 0.524822695035461
AUC_ROC
Test set: 0.5165006226650062


### Parámetro class_weight

#### Modelo regresión logística 

In [43]:
model = LogisticRegression(random_state=12345,class_weight = 'balanced', solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.6352433764633395
Test set: 0.47450980392156866
AUC_ROC
Test set: 0.5054301445720679


#### Modelo árbol de decisión

In [57]:
model = DecisionTreeClassifier(random_state=12233, class_weight = 'balanced', max_depth = 4)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.560145808019441
Test set: 0.5534351145038168
AUC_ROC
Test set: 0.5154820754657345


#### Modelo bosque aleatorio 

In [78]:
model = RandomForestClassifier(random_state=12233, class_weight = 'balanced', min_samples_split = 17, n_estimators=65)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.9651898734177214
Test set: 0.6090534979423868
AUC_ROC
Test set: 0.5294264528323356


### Sobremuestreo

#### Modelo regresión logística 

In [46]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.5716954694068193
Test set: 0.4268551236749117
AUC_ROC
Test set: 0.5061302476120639


#### Modelo árbol de decisión

In [47]:
model = DecisionTreeClassifier(random_state=12233, max_depth = 3)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.3452840595697738
Test set: 0.32705980761187786
AUC_ROC
Test set: 0.5241875327926366


#### Modelo bosque aleatorio

In [59]:
model = RandomForestClassifier(random_state=12233, min_samples_split = 11,n_estimators=37)
model.fit(features_upsampled, target_upsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.9992019154030327
Test set: 0.5836385836385836
AUC_ROC
Test set: 0.5246102812422957


### Submuestreo

#### Modelo regresión logística 

In [49]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.4369108049311095
Test set: 0.39799331103678925
AUC_ROC
Test set: 0.5070721469615844


#### Modelo árbol de decisión

In [50]:
model = DecisionTreeClassifier(random_state=12233, max_depth = 4)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.4475308641975308
Test set: 0.42957746478873243
AUC_ROC
Test set: 0.5241835818725464


#### Modelo bosque aleatorio

In [60]:
model = RandomForestClassifier(random_state=12233, min_samples_split = 2, n_estimators=4)
model.fit(features_downsampled, target_downsampled)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('F1')
print('Training set:', f1_score(target_train, train_predictions))
print('Test set:', f1_score(target_test, test_predictions))
print('AUC_ROC')
print('Test set:', auc_roc)

F1
Training set: 0.5137263247499468
Test set: 0.4259140474663246
AUC_ROC
Test set: 0.5142588706057867


El modelo con la métrica deseada fue bosque aleatorio con el parámetro class_weight en balanced.

## Conclusión <a id='conclusion'></a>

En conclusión, el mejor modelo para el conjunto de datos fue el bosque aleatorio en el cual se experimentó con los hiperparámetros, min_samples_split y n_estimators, ya que con la evaluación de solo un hiperparametro no se pudo tener el valor F1 deseado. Y la técnica que mejor resultado dio para el equilibrio de clases fue el parámetro class_weight en balanced.
- Los hiperparámetros afectaron la calidad de predidcción.
- Se obtuvo un valor F1 mayor a 0.59 en el conjunto de validación y de prueba.
- El valor AUC-ROC fue mayor a la mitad a 0.5.


<div class="alert alert-block alert-success">
 
# Comentarios generales
<b>Comentario del revisor</b> <a class="tocSkip"></a>
    
Espectacular trabajo, Diana. Has cumplido con todos los requerimientos y has aprobado un nuevo proyecto. ¡Felicitaciones!
</div>